In [9]:
import torch
import torch.nn as nn # For building NN layers
import torch.optim as optim # For optimization algorithms (Adam, SGD, etc)
import torchvision.transforms as transforms # For preprocessing images
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as pd
import matplotlib.pyplot as plt
from PIL import Image



In [2]:
class CNN (nn.Module):
    def __init__ (self):
        super(CNN, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(1, 32, kernel_size=3, 
                                     stride=1, padding=1)
        self.cnn_layer_2 = nn.Conv2d(32, 64, kernel_size=3, 
                                     stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128) # 64 filters, image size = 7x7 after pooling
        self.fc2 = nn.Linear(128, 10) # 10 output classes for MNIST digts (0-9)
        
    def forward(self, x):
        x = self.pool(F.relu(self.cnn_layer_1(x)))
        x = self.pool(F.relu(self.cnn_layer_2(x)))
        x = x.view(-1, 64 * 7 * 7) # Flatten the tensor for fully connected layers
        x = F.relu(self.fc1(x)) # Apply ReLU activation to the first FC layer
        x = self.fc2(x) # Apply the second FC layer
        return x

Create the model

In [3]:
model = CNN()

Define the loss and optimizer functions

In [4]:
criterion = nn.CrossEntropyLoss() # Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001) # Optimizer

Prepare the dataset

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5))])

trainset = MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


Train the model

In [7]:
model.train() # set model to training mode
for images, labels in trainloader:
    optimizer.zero_grad() # Zero the gradients to prevent accumulation of gradients from previous iterations
    outputs = model(images) # Forward pass
    loss = criterion(outputs, labels) # Calculate the loss
    loss.backward() # Backward pass
    optimizer.step() # Update model parameters

Evaluate  model on test set

In [8]:
model.eval() # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Inference mode, gradients aren't needed
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # Get the index of the max log probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the networks on the 10,000 test images: %d %%' % (100 * correct / total))



Accuracy of the networks on the 10,000 test images: 98 %
